In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores("global warming")

{'neg': 0.0, 'neu': 0.385, 'pos': 0.615, 'compound': 0.1531}

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
from tqdm.auto import tqdm
tqdm.pandas()
from pprint import pprint
import json

files = sorted(glob.glob("data/from_research_API/*.jsonl"))
print(len(files), files[0], files[-1])

1933 data/from_research_API/2013-01-30.jsonl data/from_research_API/2021-06-21.jsonl


In [3]:
with open("data/from_research_API/2013-01-30.jsonl") as f:
    obj = json.loads(f.readline())

In [4]:
obj.keys()

dict_keys(['data', 'includes', 'errors', 'meta', '__twarc'])

In [5]:
pprint(obj["data"][0])

{'author_id': '51828412',
 'conversation_id': '296769715127336961',
 'created_at': '2013-01-30T23:59:48.000Z',
 'entities': {'annotations': [{'end': 42,
                               'normalized_text': 'MN',
                               'probability': 0.908,
                               'start': 41,
                               'type': 'Place'}],
              'hashtags': [{'end': 140, 'start': 126, 'tag': 'climatechange'}],
              'mentions': [{'end': 18,
                            'id': '22819917',
                            'start': 3,
                            'username': 'wildlifeaction'},
                           {'end': 125,
                            'id': '3554721',
                            'start': 121,
                            'username': 'NWF'}],
              'urls': [{'display_url': 'bit.ly/14vlp6k',
                        'end': 116,
                        'expanded_url': 'http://bit.ly/14vlp6k',
                        'start': 96,
         

In [6]:
results = []
for filename in tqdm(files):
    with open(filename) as file:
        for line in file:
            obj = json.loads(line)
            for tweet in obj["data"]:
                result = analyzer.polarity_scores(tweet["text"])
                result["id"] = tweet["id"]
                result["created_at"] = tweet["created_at"]
                results.append(result)
print(len(results))

  0%|          | 0/1933 [00:00<?, ?it/s]

79884202


In [9]:
df = pd.DataFrame(results)
df

,neg,neu,pos,compound,id,created_at
0,0.000,1.000,0.000,0.0000,296769715127336961,2013-01-30T23:59:48.000Z
1,0.000,0.738,0.262,0.5994,296769710094168065,2013-01-30T23:59:47.000Z
2,0.230,0.628,0.141,-0.4019,296769692549410819,2013-01-30T23:59:43.000Z
3,0.209,0.664,0.128,-0.4019,296769668813828097,2013-01-30T23:59:37.000Z
4,0.000,1.000,0.000,0.0000,296769618310205440,2013-01-30T23:59:25.000Z
...,...,...,...,...,...,...
79884197,0.138,0.862,0.000,-0.6800,1406763810761940992,2021-06-21T00:00:03.000Z
79884198,0.000,0.925,0.075,0.2732,1406763802369085440,2021-06-21T00:00:01.000Z
79884199,0.000,0.949,0.051,0.2732,1406763801865826306,2021-06-21T00:00:01.000Z
79884200,0.000,0.882,0.118,0.3400,1406763797440782336,2021-06-21T00:00:00.000Z


In [11]:
df.to_csv("tweets_from_research_API_sentiment2.csv", index=False)